In [72]:
import boto3
import os
from config import ACCESS_KEY, SECRET_KEY, TOKEN
import datetime
import pytz

import pandas as pd
pd.set_option('display.max_colwidth', None)

token = TOKEN
headers={'Authorization':token,'Accept':'application/json'}

#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
ACCESS_KEY = ACCESS_KEY #aws_access_key_id для S3
SECRET_KEY = SECRET_KEY #aws_secret_access_key в s3

bucket_name = 'dwh-asgard' # имя бакета

In [73]:
#создаем подключение к yndex storage через .session.Session()

def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )

s3 = get_s3_instance()

In [74]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")

# Example usage
folder_path = 'C:/Users/Boris/Desktop/работа/new_folder'
create_folder_if_not_exists(folder_path)

Folder 'C:/Users/Boris/Desktop/работа/new_folder' already exists.


In [ ]:
path_on_pc = 'C:/Users/Boris/Desktop/работа/files_for_rework/' 
date = '2024-01-11'

path = datetime.datetime.strptime(date,'%Y-%m-%d').strftime('year=%Y/month=%m/%d.csv')
create_folder_if_not_exists(path)

# entries_df_20 = pd.read_csv(f'{path_on_pc}/entries/{path}')
# entries_df_20[entries_df_20['subsc_restrict'] == 'freemonetization']['subsc_restrict'].count()

In [ ]:
# Если нужны все файлы, то не прописываем Prefix то Prefix=False
def list_of_daily_objects_from_s3(bucket_name, Prefix):
    s3 = get_s3_instance()
    # Создаем пагинатор. Он нужен на тот случай, если файлов больше 1000
    paginator = s3.get_paginator('list_objects_v2')

    # Присваеваем пагинатор с параметрами. 
    if folder is False:
        page_iterator = paginator.paginate(Bucket=bucket_name)
    else:    
        page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=Prefix)

    df_obj = pd.DataFrame({
        'folder': [],
        'year': [],
        'month': [],
        'day': [],
        'key':[],
        'LastModified':[],
        }) # создаем пустой датасет для анализа

    df_obj = df_obj.astype(str)

    obj_number = 0 
    for page in page_iterator:
        for obj in page['Contents']: # list_objects - список объектов в бакете, Prefix - поиск во ключевому слову
            if len(obj['Key'].split(sep='/')) == 4 \
            and obj['Key'].split(sep='/')[3]!='':   # условия фильтрации
                df_obj.loc[obj_number,['folder']] = obj['Key'].split(sep='/')[0]
                df_obj.loc[obj_number,['year']] = obj['Key'].split(sep='/')[1].replace('year=','')
                df_obj.loc[obj_number,['month']] = obj['Key'].split(sep='/')[2].replace('month=','')
                df_obj.loc[obj_number,['day']] = obj['Key'].split(sep='/')[3].replace('.csv','').replace('.lz4','')
                df_obj.loc[obj_number,['key']] = obj['Key']
                df_obj.loc[obj_number,['LastModified']] = obj['LastModified'].strftime('%Y-%m-%d %H:%M:%S',)
            obj_number += 1 
                # добавляем в датасет найденные значения
                
    df_obj['date'] = df_obj['year']+'-'+df_obj['month']+'-'+df_obj['day']
    df_obj['date'] = pd.to_datetime(df_obj['date'], dayfirst=False)

    df_obj = df_obj.reset_index().drop('index', axis=1)
    return df_obj

# В качестве ключей для скачивания используем ключи всех файлов из папки, полученные ранее

def downloading_dayly_files_from_s3(path_on_pc,df_obj):
    s3 = get_s3_instance()
    for i in range(0,df_obj.shape[0]):
        s3_file_name = df_obj.loc[i,['key']].values[0]
        print(s3_file_name)
        s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])
        
        local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
        print(local_download_folder)
        local_download_path = f'{path_on_pc}/{s3_file_name}'
        print(local_download_path)
        
    # если данный файл уже скачан, он не скачивается повторно
        if os.path.isfile(local_download_path) is False:
        # создается новая папка, если ее нет
            os.makedirs(local_download_folder, exist_ok=True)
            s3.download_file(Bucket=bucket_name,Key=s3_file_name,Filename=local_download_path)


In [109]:
folder = 'companies_st_partner' # имя папки в бакете
Prefix = f"{folder}/"
df_obj = list_of_daily_objects_from_s3(bucket_name,Prefix)
df_obj

,folder,year,month,day,key,LastModified,date
0,companies_st_partner,2023,10,1,companies_st_partner/year=2023/month=10/1.csv,2023-10-02 01:00:19,2023-10-01
1,companies_st_partner,2023,10,10,companies_st_partner/year=2023/month=10/10.csv,2023-10-11 01:00:54,2023-10-10
2,companies_st_partner,2023,10,11,companies_st_partner/year=2023/month=10/11.csv,2023-10-12 01:00:59,2023-10-11
3,companies_st_partner,2023,10,12,companies_st_partner/year=2023/month=10/12.csv,2023-10-13 01:01:11,2023-10-12
4,companies_st_partner,2023,10,13,companies_st_partner/year=2023/month=10/13.csv,2023-10-14 01:00:30,2023-10-13
...,...,...,...,...,...,...,...
557,companies_st_partner,2025,2,5,companies_st_partner/year=2025/month=2/5.csv,2025-02-06 04:00:14,2025-02-05
558,companies_st_partner,2025,2,6,companies_st_partner/year=2025/month=2/6.csv,2025-02-07 04:00:33,2025-02-06
559,companies_st_partner,2025,2,7,companies_st_partner/year=2025/month=2/7.csv,2025-02-08 04:00:35,2025-02-07
560,companies_st_partner,2025,2,8,companies_st_partner/year=2025/month=2/8.csv,2025-02-09 04:00:10,2025-02-08


In [110]:
df_obj['date'] = pd.to_datetime(df_obj['date'])
df_obj = df_obj.sort_values(['date']).reset_index(drop=True)
df_obj = df_obj[0:466]

In [111]:
df_obj
df_obj.to_csv('test2.csv')

In [67]:
list_of_references = ['f1f4d289-beb1-44ad-954b-42a8f02364e3',
                        'ba308431-fd0c-4921-a605-9e21c52f8ac6',
                        '7775bc0f-24e5-4afc-9054-e194c847049b',
                        'd950900f-ea1a-4c6e-90d2-fe83ebf8feac']

In [ ]:
df_for_changing = pd.read_csv(f'C:/Users/Boris/Desktop/работа/files_for_rework/{df_obj.loc[100,'key']}')

In [ ]:
df_for_changing[df_for_changing['partner_uuid'].isin(list_of_references)]

In [ ]:
len(range(0,1))

15

In [124]:
partner_uuid

'd950900f-ea1a-4c6e-90d2-fe83ebf8feac'

In [141]:
df_for_changing = pd.read_csv(f'{path_on_pc}/{df_obj.loc[i,'key']}')

In [ ]:
df_for_changing['enterprise_not_paid'].fillna(0)

In [146]:
df_for_changing['enterprise_not_paid'].fillna(0)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
5238    0.0
5239    0.0
5240    0.0
5241    0.0
5242    0.0
Name: enterprise_not_paid, Length: 5243, dtype: float64

In [175]:
df_obj[df_obj['date'] == '2023-11-01']

,folder,year,month,day,key,LastModified,date
93,companies_st_partner,2023,11,1,companies_st_partner/year=2023/month=11/1.csv,2023-11-02 00:02:18,2023-11-01


In [181]:
path_on_pc = 'C:/Users/Boris/Desktop/работа/files_for_rework/'
path_on_pc_2 = 'C:/Users/Boris/Desktop/работа/files_for_rework/reworked'
for i in list(range(0,df_obj.shape[0])):
    df_for_changing = pd.read_csv(f'{path_on_pc}/{df_obj.loc[i,'key']}')
    for partner_uuid in list_of_references:
        index1 = df_for_changing[df_for_changing['partner_uuid'] == partner_uuid].index[0]
        df_for_changing.loc[index1,'enterprise_not_paid'] =  1
        df_for_changing.loc[index1,'enterprise_subs'] =  1
    
    df_for_changing['enterprise_subs']= df_for_changing['enterprise_subs'].fillna(0).astype(int)
    df_for_changing['enterprise_not_paid'] = df_for_changing['enterprise_not_paid'].fillna(0).astype(int)
    df_for_changing['pro_subs'] = df_for_changing['pro_subs'].fillna(0).astype(int)
    df_for_changing['is_blocked'] = df_for_changing['is_blocked'].fillna(0).astype(int)
    df_for_changing['billing_pro'] = df_for_changing['billing_pro'].fillna(0).astype(int)

    s3_file_name = df_obj.loc[i,['key']].values[0]
    s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])
    local_download_folder = f'{path_on_pc_2}/{s3_file_name_folder}'
    local_download_path = f'{path_on_pc_2}/{s3_file_name}'
    if os.path.isfile(local_download_path) is False:
        # создается новая папка, если ее нет
        os.makedirs(local_download_folder, exist_ok=True)
        df_for_changing.to_csv(local_download_path, index=False)
    else:
        df_for_changing.to_csv(local_download_path, index=False)    

In [156]:
df_for_changing['enterprise_not_paid'] = df_for_changing['enterprise_not_paid'].fillna(0).astype(int)

In [182]:
path_on_pc_2 = 'C:/Users/Boris/Desktop/работа/files_for_rework/reworked'
for i in range(93,df_obj.shape[0]):
    key = df_obj.loc[i,['key']].values[0]
    s3_file_path = f'{key}'
    pc_file_path = f'{path_on_pc_2}/{key}'
    try:
        s3.upload_file(pc_file_path, bucket_name, s3_file_path)
        print(f'{key} has been uploaded')
    except Exception as e:
        print(e)
        continue

companies_st_partner/year=2023/month=11/1.csv has been uploaded
companies_st_partner/year=2023/month=11/2.csv has been uploaded
companies_st_partner/year=2023/month=11/3.csv has been uploaded
companies_st_partner/year=2023/month=11/4.csv has been uploaded
companies_st_partner/year=2023/month=11/5.csv has been uploaded
companies_st_partner/year=2023/month=11/6.csv has been uploaded
companies_st_partner/year=2023/month=11/7.csv has been uploaded
companies_st_partner/year=2023/month=11/8.csv has been uploaded
companies_st_partner/year=2023/month=11/9.csv has been uploaded
companies_st_partner/year=2023/month=11/10.csv has been uploaded
companies_st_partner/year=2023/month=11/11.csv has been uploaded
companies_st_partner/year=2023/month=11/12.csv has been uploaded
companies_st_partner/year=2023/month=11/13.csv has been uploaded
companies_st_partner/year=2023/month=11/14.csv has been uploaded
companies_st_partner/year=2023/month=11/15.csv has been uploaded
companies_st_partner/year=2023/mon